# 0. Importação de Biblíotecas e Funções de Ajuda

## 0.1 Bibliotecas

In [1]:
import joblib
import pandera

import pandas as pd

from IPython.display               import display, HTML
from pandera                       import Check, Column, DataFrameSchema
from sklearn.model_selection       import train_test_split
from sklearn.pipeline              import Pipeline
from sklearn.preprocessing         import RobustScaler, StandardScaler
from sklearn.metrics               import roc_auc_score
from sklearn.linear_model          import LogisticRegression
from feature_engine.discretisation import EqualFrequencyDiscretiser, EqualWidthDiscretiser
from feature_engine.imputation     import MeanMedianImputer
from feature_engine.wrappers       import SklearnTransformerWrapper

## 0.2 Funções de Ajuda

In [2]:
columns_to_use = ['target',
                  'TaxaDeUtilizacaoDeLinhasNaoGarantidas',
                  'Idade',
                  'NumeroDeVezes30-59DiasAtrasoNaoPior',
                  'TaxaDeEndividamento',
                  'RendaMensal',
                  'NumeroDeLinhasDeCreditoEEmprestimosAbertos',
                  'NumeroDeVezes90DiasAtraso',
                  'NumeroDeEmprestimosOuLinhasImobiliarias',
                  'NumeroDeVezes60-89DiasAtrasoNaoPior',
                  'NumeroDeDependentes']

## 0.3 Configurações do Jupyter

In [3]:
# Deixar o jupyper em widescreen
display(HTML("<style>.container { width:90% !important; }</style>"))

# Seta o máximo de colunas e linhas que o pandas vai exibir
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 60)

# 1. Carregamento de Dados

In [4]:
class DataLoad:
    """Classe responsável pelo carregamento dos dados"""
    
    def __init__(self) -> None:
        pass
    
    def load_data(self) -> pd.DataFrame:
        """Esta função vai retornar os dados carregados
        
            return:
                pandas.DataFrame"""
        
        dataframe = pd.read_csv('../dataset/raw/train.csv', index_col=0)
        return dataframe

In [5]:
df = DataLoad().load_data()

# 2. Validação dos Dados

In [6]:
class DataValidation:
    """Classe responsável pela validação dos dados"""
    
    def __init__(self, columns_to_use) -> None:
        self.columns_to_use = columns_to_use
        
    def check_data_shape(self, dataframe: pd.DataFrame) -> bool:
        """Esta função checa se todas as colunas estão presentes no DataFrame"""
        try:
            print('A Validação Começou')
            dataframe.columns = self.columns_to_use
            return True
        except Exception as e:
            print(f'A Validação Falhou \n {e}')
            return False
    
    def check_columns(self, dataframe: pd.DataFrame) -> bool:
        """Esta função checa se os tipos de dados são os esperados"""
        schema = DataFrameSchema(
            {
            "target": Column(int, Check.isin([0,1 ]), Check(lambda x: x>0), coerce=True),
            "TaxaDeUtilizacaoDeLinhasNaoGarantidas": Column(float, nullable=True),
            "Idade": Column(int, nullable=True),
            "NumeroDeVezes30-59DiasAtrasoNaoPior": Column(int, nullable=True),
            "TaxaDeEndividamento": Column(float, nullable=True),
            "RendaMensal": Column(float, nullable=True),
            "NumeroDeLinhasDeCreditoEEmprestimosAbertos": Column(int, nullable=True),
            "NumeroDeVezes90DiasAtraso": Column(int, nullable=True),
            "NumeroDeEmprestimosOuLinhasImobiliarias": Column(int, nullable=True),
            "NumeroDeVezes60-89DiasAtrasoNaoPior": Column(int, nullable=True),
            "NumeroDeDependentes": Column(float, nullable=True)
            }
        )
        try:
            schema.validate(dataframe)
            print('Validação das colunas passou')
            return True
        except Exception as e:
            print(f"Validação das colunas falhou \n {e}")
            return False
    def run(self, dataframe: pd.DataFrame) -> bool:
        if self.check_data_shape(dataframe) and  self.check_columns(dataframe):
            print('Validação concluida com sucesso!')
            return True
        else:
            print('Validação falhou')
            return False

In [7]:
dv = DataValidation(columns_to_use).run(df)
dv

A Validação Começou
Validação das colunas passou
Validação concluida com sucesso!


True

# 3. DataTransformation

In [86]:
class DataTransformation:
    """Classe responsável pela transformação dos dados"""
    def __init__(self, 
                 dataframe: pd.DataFrame,
                 target_name: str):
        self.dataframe = dataframe
        self.target_name = target_name
    
    def train_test_spliting(self, t_size: float):
        """Esta função separa os dados para treinamento e teste do modelo"""
        X = self.dataframe.drop(columns=self.target_name)
        y = self.dataframe[self.target_name].values
        
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = t_size, stratify=y)
        
        return X_train, X_val, y_train, y_val

In [87]:
X_train, X_val, y_train, y_val = DataTransformation(df, 'target').train_test_spliting(t_size=0.2)

# 4. Pre-Processamento dos dados

In [88]:
class DataPreprocess:
    def __init__(self, 
                 dataframe: pd.DataFrame,
                 pipe: Pipeline):
        self.dataframe = dataframe
        self.pipe = pipe
        
    def pipeline(self):
        train_pipe = self.pipe
        train_pipe.fit(self.dataframe)
        return train_pipe
    
    def run(self):
        print('Pre-processador iniciou')
        treined_pipe = self.pipeline()
        data_preprocessed = treined_pipe.transform(self.dataframe)
        print('Pre-processador finalizou')
        return data_preprocessed

In [89]:
pipe = Pipeline([('imputer',  MeanMedianImputer(variables=['RendaMensal', 
                                                           'NumeroDeDependentes'])),
                  ('discretizer', EqualFrequencyDiscretiser(variables=['TaxaDeUtilizacaoDeLinhasNaoGarantidas',
                                                                      'TaxaDeEndividamento',
                                                                      'RendaMensal'])),
                  ('scaler', SklearnTransformerWrapper(StandardScaler()))
                ])

In [90]:
dp = DataPreprocess(X_train, pipe)

In [91]:
X_train_processed = dp.run()

Pre-processador iniciou
Pre-processador finalizou


In [92]:
joblib.dump(dp.pipeline(), 'preprocessador.joblib')

['preprocessador.joblib']

# 5. Treinamento do Modelo

In [99]:
class TrainModel:
    def __init__(self, 
                 dados_x: pd.DataFrame,
                 dados_y: pd.DataFrame):
        self.dados_x = dados_x
        self.dados_y = dados_y
        
    def train(self, model):
        model.fit(self.dados_x, self.dados_y)
        joblib.dump(model, 'modelo.joblib')
        return model
    
    def predict(self, dados_para_prever: pd.DataFrame):
        model_fitted = self._load_model()
        dados_pred = model_fitted.predict_proba(dados_para_prever)
        return dados_pred
    
    def _load_model(self):
        model = joblib.load('modelo.joblib')
        return model

In [100]:
tm = TrainModel(dados_x = X_train_processed,
                dados_y = y_train)

In [101]:
tm.train(model=LogisticRegression())

LogisticRegression()

In [102]:
y_train_pred = tm.predict(X_train_processed)

In [104]:
y_train_pred

array([[0.74250278, 0.25749722],
       [0.93813324, 0.06186676],
       [0.86722592, 0.13277408],
       ...,
       [0.94037466, 0.05962534],
       [0.88630675, 0.11369325],
       [0.91738007, 0.08261993]])

# 6. Avaliação do Modelo

In [105]:
preprocessador = dp.pipeline()

In [106]:
preprocessador

Pipeline(steps=[('imputer',
                 MeanMedianImputer(variables=['RendaMensal',
                                              'NumeroDeDependentes'])),
                ('discretizer',
                 EqualFrequencyDiscretiser(variables=['TaxaDeUtilizacaoDeLinhasNaoGarantidas',
                                                      'TaxaDeEndividamento',
                                                      'RendaMensal'])),
                ('scaler',
                 SklearnTransformerWrapper(transformer=StandardScaler()))])

In [107]:
X_val_processed = preprocessador.transform(X_val)

In [117]:
y_val_pred = tm.predict(X_val_processed)[:, 1]

In [118]:
y_val_pred

array([0.0170523 , 0.01397957, 0.0680838 , ..., 0.04212155, 0.02303131,
       0.07864578])

In [110]:
class ModelEvaluation():
    def __init__(self):
        pass
    
    def eval_Metrics(self, dados_reais, dados_preditos):
        roc_auc = roc_auc_score(dados_reais, dados_preditos)
        return roc_auc
        

In [111]:
me = ModelEvaluation()

In [119]:
me.eval_Metrics(dados_reais=y_val, dados_preditos=y_val_pred)

0.7896209289243404